Краткий аутлайн решения:
- Кластеризуем по каждому спутнику по реальным координатам, надеемся что получатся хорошие кластеры
- Высчитываем MEAN, Median, чето еще по кластеру (точку)
- Точки из SGP4 относим к кластеру
- В первом приближении вместо SGP4 предсказания присваваем эту среднюю точку, смотрим что будет
- Во втором приближении предсказываем что выбрать, SGP4 или эту среднюю точку, мб взять несколько точек по такому кластеру или еще что-то

<b> Такое решение придумано на тот случай если всякие RNN и тому подобное не зайдут совсем, а такое вполне возможно.. </b>

Большой привет соревнованию ML Bootcamp, надо взять еще идей оттуда

P.S можно попробовать попридумывать альтернативные таргеты, так же как делали в ml bootcamp

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

%matplotlib inline

PATH = '../'

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.59 s


In [3]:
test_unchanged = test.copy()
train_unchanged = train.copy()

### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [4]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

In [5]:
def getLBScore(real, predicted):
    smp = []
    for sat_id in real['sat_id'].unique():
        rv = real[real['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        sym = predicted[predicted['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        smp.append(smape(sym.ravel(), rv.ravel()))
    meanSmape = np.mean(smp)
    lbScore = 100 * (1 - meanSmape)
    print('Mean SMAPE ', meanSmape, '; LB score ', lbScore)

### Clustering

In [256]:
import plotly.express as px
import plotly.graph_objects as go

In [288]:
SAT_ID = np.random.choice(train['sat_id'].unique(), 1)[0]
satTrain = train[train['sat_id'] == SAT_ID]
satTest = test[test['sat_id'] == SAT_ID]

print(f'ID: {SAT_ID}, Total observations: {satTrain.shape[0]}; In test {satTest.shape[0]}')

ID: 59, Total observations: 2583; In test 2327


In [289]:
x,y,z = satTrain.loc[:, 'x'].values, satTrain.loc[:, 'y'].values, satTrain.loc[:, 'z'].values
xSim,ySim,zSim = satTrain.loc[:, 'x_sim'].values, satTrain.loc[:, 'y_sim'].values, satTrain.loc[:, 'z_sim'].values

xTestSim, yTestSim, zTestSim = satTest.loc[:, 'x_sim'].values, satTest.loc[:, 'y_sim'].values, satTest.loc[:, 'z_sim'].values
fig1 = go.Scatter3d(x=x, 
                    y=y, 
                    z=z, 
                    mode='markers', 
                    marker=dict(
                        #color='rgb(127, 127, 127)',
                        color = np.arange(satTrain.shape[0]),
                        colorscale = 'Viridis',
                        showscale = True,
                        size=8, 
                        symbol='circle', 
                        line=dict(
                            color='rgb(204, 204, 204)',
                            width=0.5),
                        opacity=0.9),
                   name = 'Real')
fig2 = go.Scatter3d(x=xSim, 
                    y=ySim, 
                    z=zSim, 
                    mode = 'markers', 
                    marker=dict(
                        size=8,
                        line=dict(
                            color='rgba(217, 217, 217, 0.14)',
                            width=0.5
                        ),
                    opacity=0.8),
                    name = 'Simulated')

fig3 = go.Scatter3d(x=xTestSim, 
                    y=yTestSim, 
                    z=zTestSim, 
                    mode = 'markers', 
                    marker=dict(
                        size=8,
                        line=dict(
                            color='rgba(255, 87, 123, 0.05)',
                            width=0.5
                        ),
                    opacity=0.8),
                    name = 'SimulatedTest')
fig = go.Figure(data = [fig1, fig2, fig3])
fig.show()

In [274]:
from sklearn.cluster import DBSCAN, KMeans, MeanShift
from sklearn.preprocessing import StandardScaler, MinMaxScaler
model = DBSCAN()
labels = model.fit_predict(MinMaxScaler().fit_transform(satTrain))
print('Num unique labels: ', len(np.unique(labels)))

ValueError: could not convert string to float: '2014-01-01T00:00:00.000'

In [209]:
fig1 = go.Scatter3d(x=x,
                    y=y,
                    z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=labels,  # set color equal to a variable
                        colorscale='Viridis',  # one of plotly colorscales
                        showscale=True
                    ),
                    name='Real')
fig = go.Figure(data = [fig1])
fig.show()

In [153]:
import numpy as np

fig = go.Figure(data=go.Scatter(
    x = x,
    y = y,
    mode='markers',
    marker=dict(
        size=15,
        color=labels, #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

fig.show()